In [1]:
import torch
import torch.nn as nn

In [44]:
lenet = nn.Sequential(nn.Conv2d(1,6, kernel_size=5), nn.BatchNorm2d(6), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Conv2d(6,16, kernel_size=5), nn.BatchNorm2d(16), nn.ReLU(),nn.MaxPool2d(kernel_size=2,stride=2),
                      nn.Flatten(), 
                      nn.Linear(256, 120), nn.BatchNorm1d(120), nn.ReLU(),
                      nn.Linear(120,64), nn.BatchNorm1d(64), nn.ReLU(),
                      nn.Linear(64,6))

In [45]:
X = torch.randn(10,1,28,28)
def look_in_net(net, X):
    out = X
    for layer in net:
        out = layer(out)
        print(f"{layer.__class__.__name__} : {out.shape}")

look_in_net(lenet, X)

Conv2d : torch.Size([10, 6, 24, 24])
BatchNorm2d : torch.Size([10, 6, 24, 24])
ReLU : torch.Size([10, 6, 24, 24])
MaxPool2d : torch.Size([10, 6, 12, 12])
Conv2d : torch.Size([10, 16, 8, 8])
BatchNorm2d : torch.Size([10, 16, 8, 8])
ReLU : torch.Size([10, 16, 8, 8])
MaxPool2d : torch.Size([10, 16, 4, 4])
Flatten : torch.Size([10, 256])
Linear : torch.Size([10, 120])
BatchNorm1d : torch.Size([10, 120])
ReLU : torch.Size([10, 120])
Linear : torch.Size([10, 64])
BatchNorm1d : torch.Size([10, 64])
ReLU : torch.Size([10, 64])
Linear : torch.Size([10, 6])


In [46]:
import torchvision

my_transforms = torchvision.transforms.Compose(
                [
                    torchvision.transforms.ToTensor(),
                ]
)

In [48]:
#train_dataset = torchvision.datasets.Cityscapes( root="../../data", transform=my_transforms)
#test_dataset = torchvision.datasets.Cityscapes(root="../../data", transform=my_transforms)
from torch.utils.data import DataLoader
from torchvision import datasets

batch_size=64

train_dataset = datasets.FashionMNIST(download=False, root="../../data", train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(download=False, root="../../data", train=False, transform=my_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [49]:
for X,y in train_dataloader:
    print(X.shape, len(y))
    break

torch.Size([64, 1, 28, 28]) 64


In [50]:
net = lenet
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1)

In [51]:
num_epochs = 10

def accuracy(y_hat, y):
    print(torch.argmax(y_hat, dim=1))
    
    return (torch.argmax(y_hat, dim=1)==y).sum().float().mean()

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
net = net.to(device)
for epoch in num_epochs:
    for X, y in train_dataloaders:
        X = X.to(device)
        y = y.to(device)
        
        y_hat = net(X)
        
        loss = loss_criterion(y_hat,y)
        
        current_acc = accuracy(y_hat, y)
        
        print(current_acc)
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

TypeError: 'int' object is not iterable